# Cable Unidimensional con impureza (caso TD)

Importamos los módulos que necesitamos, para dibujar y hacer álgebra matricial

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import math
%matplotlib inline

Trabajamos en unidades atómicas pero nos gusta que aparezcan las constantes en los lugares que esperamos, hacemos $\hbar =1$

In [ ]:
hbar = 1.0

Parámetros generales de la simulación que definen el sistema:
* `nsites` es la longitud del cable en sitios, lo hacemos impar por conveniencia, esto nos permite poner una impureza en el medio.
* `temp` es la temperatura en unidades de $k_b T$, la usamos para construír la matriz densidad inicial.
* `mu` es el potencial electroquímico de los electrones para el sistema en equilibrio.
* `beta` es el hopping entre sitios en el cable $\beta$

In [ ]:
nsites = 101
temp = 0.01
mu = 0.0
beta = -1.0

Parámetros generales que definen **esta** simulación:
* `nsteps` es el número de pasos que queremos correr.
* `dt` es el paso de tiempo $\Delta t$.
* `DeltaV` es la diferencia de potencial inicial (bias) $\Delta V$

In [ ]:
nsteps = 10000
dt = 0.01
DeltaV = 1.1

Definimos la $\delta$ de Kronecker

In [ ]:
def kdelta(k, l):
    if k == l: return True
    else: return False

Construímos el Hamiltoniano con el que armaremos el estado inicial del sistema, es tridiagonal, y tiene incorporado el bias ($\Delta V$), a la izquierda subimos el potencial local de los sitios en $\frac{\Delta V}{2}$ y a la derecha lo bajamos en $\frac{\Delta V}{2}$. Dejamos nuestra "impureza" sin potencial. **IMPORTANTE** Esta forma de armar el Hamiltoniano no está escrita para que sea eficiente (y mucho menos en Python) sino para que sea fácil de leer. Nótese por ejemplo que recorremos **toda** la matriz para hacer sólo unos muy poquitos elementos distintos de cero. En la "vida real" uno haría las cosas de otra forma mejor, hay, como siempre, muchas posible y no entraremos en esto.

In [ ]:
h = np.zeros((nsites,nsites))

for i in range(0,nsites):
    for j in range (0,nsites):
        if kdelta(i,j-1):
            h[i,j] = beta
        elif kdelta(i,j+1):
            h[i,j] = beta
        elif kdelta(i,j) and i < int(nsites/2):
            h[i,j] = DeltaV / 2
        elif kdelta(i,j) and i > int(nsites/2):
            h[i,j] = - DeltaV / 2        

In [ ]:
int(nsites/2)

Definimos la función de Fermi 
$$f(\epsilon) = \frac{1}{1-e^{\frac{\epsilon-\mu}{k_b T}}}$$
nótese que revisamos el valor del exponente para no caer en una situación de under/overflow antes de evaluar la exponencial.

In [ ]:
def fermif(energy, mu, temp):
    expo = (energy-mu)/temp
    if expo < -100.0:
        return 1.0
    elif expo > 100.0:
        return 0.0
    else:
        return 1.0/(math.exp(expo)+1.0)

Construímos la matriz densidad del estado inicial $\hat\rho_0$. Para eso primero la armamos en la base de autofunciones de $\hat{H}$, en la que es diagonal y vale $\hat\rho_0^d = f(\hat H)$, con $f$ la función de Fermi. Para eso diagonalizamos $\hat H$, `w` contiene los autovalores y `V` la matriz de autovectores. Usamos le aplicamos $f$ a `w` para armar la $\hat\rho_0$ en la base de autofunciones de $\hat H$ y después la transformamos a la base de sitios haciendo $\hat\rho_0 = V \hat\rho_0^d V^\dagger$ 

In [ ]:
w, v = np.linalg.eig(h.real)
rho_diag = np.zeros((nsites, nsites))
for i in range(0, nsites):
    rho_diag[i, i] = fermif(w[i], mu, temp)
rho_0 = np.dot(v, np.dot(rho_diag, np.transpose(v)))

Miremos la pinta de las ocupaciones

In [ ]:
plt.scatter(w,np.diag(rho_diag))
plt.show()

Declaro una serie de arreglos vacíos que contendrán cosas que pretendo ir guardando durante la simulación, el tiempo, la traza, energía, corriente, etc...

In [ ]:
time = []
trace = []
energy = []
current_in = []
current_out = []

# Integración

El algoritmo de integración que usamos es el siguiente:
$$ \rho(t+\Delta t)=\rho(t-\Delta t) + 2 \Delta t\ \dot\rho(t)$$
con
$$ \dot\rho(t) = -\frac{i}{\hbar} [H,\rho] $$
el algoritmo es un *leapfrog* que usa info del paso anterior y la derivada en el presente para encontrar la $\rho$ futura. Para inicializarlo hacemos un paso de Euler hacia atrás
$$\rho_{\mathrm old} = \rho_0 - \Delta t\ \dot\rho(t=0)$$

In [ ]:
rho = rho_0
rhodot = ((0-1.0j)/hbar)*(np.dot(h,rho)-np.dot(rho,h))
rhoold = rho - dt*rhodot

Hasta ahora nuestra $\rho$ conmuta (por construcción) con el $H$ que armamos con bias, para lograr que evolucione armamos un nuevo Hamiltoniano eliminando el bias. Con esto tenemos un cable con un exceso de electrones del lado izquierdo y un defecto en el lado derecho.Con este nuevo Hamiltoniano deberíamos generar una corriente hacia la derecha. 

In [ ]:
np.fill_diagonal(h,0.0)

Hacemos evolucionar $\rho$ por `nsteps` guardando a cada paso la información que nos interese. De nuevo este código no es la forma más elegante ni eficiente, está escrito para que sea fácil de entender.

In [ ]:
for step in range(0,nsteps):
    rhodot = ((0-1.0j)/hbar)*(np.dot(h,rho)-np.dot(rho,h))
    rhonew = rhoold + 2.0 * dt * rhodot
    rhoold = rho
    rho = rhonew
    # calculate observables
    time.append(step*dt)
    trace.append(2.0*np.trace(rho).real)
    energy.append(np.trace(np.dot(h,rho)).real)
    current_in.append(4.0 * h[int(nsites/2)-1,int(nsites/2)].real * rho[int(nsites/2),int(nsites/2)-1].imag)
    current_out.append(4.0 * h[int(nsites/2),int(nsites/2)+1].real * rho[int(nsites/2)+1,int(nsites/2)].imag)

Nuestro algoritmo conserva la traza de forma numéricamente exacta.

In [ ]:
plt.plot(time,trace)
ax = plt.gca()
ax.ticklabel_format(useOffset=False)
plt.ylim((2.0*np.trace(rho_0)-0.000001,2.0*np.trace(rho_0)+0.000001))
plt.show()

Y también debería conservar la energía.

In [ ]:
plt.plot(time,energy)
ax = plt.gca()
ax.ticklabel_format(useOffset=False)
plt.ylim(np.trace(np.dot(h,rho)).real-0.000001,np.trace(np.dot(h,rho)).real+0.000001)
plt.show()

Y acá está ahora lo más interesante de todo! Qué pasa con la corriente?

In [ ]:
plt.plot(time,current_in)
plt.plot(time,current_out)
ax = plt.gca()
ax.ticklabel_format(useOffset=False)
plt.show()

De nuevo, no es la forma más eficiente, pero para poder jugar con los parámetros arriba, guardar los resultados y después leerlos para graficar (con este u otro programa) se puede hacer lo que sigue, por ejemplo guardemos el tiempo y la corriente para esta diferencia de potencial en archivos de texto:

In [ ]:
np.savetxt('tiempo.dat',time)
np.savetxt('I_1.1.dat',current_in)

Uno puede levantar una serie de cosas de disco y graficarlas todas juntas así:

In [ ]:
tiempos = np.loadtxt('tiempo.dat')
corriente_0_9 = np.loadtxt('I_0.9.dat')
corriente_1_0 = np.loadtxt('I_1.0.dat')
corriente_1_1 = np.loadtxt('I_1.1.dat')

In [ ]:
plt.plot(tiempos,corriente_0_9)
plt.plot(tiempos,corriente_1_0)
plt.plot(tiempos,corriente_1_1)
plt.show()

## Cosas para hacer

1. Explique porqué el gráfico de corriente en función del tiempo tiene la forma que tiene. ¿Qué ocurre si el cable es más largo? ¿Cómo cambia el transitorio?
2. Grafique la corriente en función de la diferencia de potencial. A pedal.
3. Haga que la impureza sea impura, agregue una energía de sitio o distinto acoplamiento con los leads y vea que pasa.